# Notebook as a Step using Notebooks Executor
The following sample shows how to use the notebook executor as part of a Vertex AI


## Libraries and Variables

In [4]:
!pip3 install  -U "git+https://github.com/kubeflow/pipelines.git#egg=kfp&subdirectory=sdk/python" -q
!pip3 install  -U google-cloud-pipeline-components -q
!pip3 install  -U google-cloud-aiplatform -q

In [15]:
from datetime import datetime
import google_cloud_pipeline_components as gcpc
from kfp.v2 import components, compiler, dsl
from kfp.v2.google.client import AIPlatformClient

In [7]:
# Variables
PROJECT_ID = 'mam-nooage'
REGION = 'us-central1' # This is where the KMS and Metastore resides from the project configuration

ROOT_PATH = ".."
PIPELINE_ROOT_PATH = f'gs://{PROJECT_ID}-vertex-root'
PACKAGE_PATH = 'notebook-as-a-step-sample-pipeline.json'

COMPONENT_YAML_PATH = f'{ROOT_PATH}/component.yaml'
WORKING_BUCKET_NAME = f'{PROJECT_ID}-naas'
INPUT_NOTEBOOK_FILE = f'gs://{WORKING_BUCKET_NAME}/runnables/run_create_bucket.ipynb'
OUTPUT_NOTEBOOK_FOLDER = f'gs://{WORKING_BUCKET_NAME}/outputs'

INPUT_NOTEBOOK_FILE

'gs://mam-nooage-naas/runnables/run_create_bucket.ipynb'

In [8]:
!gsutil ls "{PIPELINE_ROOT_PATH}" || gsutil mb -l "{REGION}" "{PIPELINE_ROOT_PATH}"
!gsutil cp -r "{ROOT_PATH}/samples/runnables/*" "gs://{WORKING_BUCKET_NAME}/runnables"
!gsutil ls "gs://{WORKING_BUCKET_NAME}/runnables"



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

gs://mam-nooage-vertex-root/780283066030/
Copying file://../samples/runnables/parameters.yaml [Content-Type=application/octet-stream]...
Copying file://../samples/runnables/run_create_bucket.ipynb [Content-Type=application/octet-stream]...
\ [2 files][  2.0 KiB/  2.0 KiB]                                                
Operation completed over 2 objects/2.0 KiB.                                      
gs://mam-nooage-naas/runnables/parameters.yaml
gs://mam-nooage-naas/runnables/run_create_bucket.ipynb


## Execute notebook

In [18]:
# Creates component.
notebooks_executor_op = components.load_component_from_file(COMPONENT_YAML_PATH)

In [19]:
# Converts component to a Vertex AI custom job
gcpc.experimental.custom_job.custom_job._DEFAULT_CUSTOM_JOB_CONTAINER_IMAGE = 'gcr.io/ml-pipeline/gcp-launcher:0.1.4'

custom_job_producer_op= gcpc.experimental.custom_job.custom_job.run_as_vertex_ai_custom_job(notebooks_executor_op)

AttributeError: module 'google_cloud_pipeline_components' has no attribute 'experimental'

## Prerequisites
You need to configure your project as detailed in https://cloud.google.com/vertex-ai/docs/pipelines/configure-project

In [10]:
BLOCK_PIPELINE = False
FAIL_PIPELINE = True

@dsl.pipeline(
    name="notebook-as-a-step-sample",
    pipeline_root=PIPELINE_ROOT_PATH)
def pipeline(
        project_id: str, 
        execution_id: str, 
        input_notebook:str, 
        output_notebook_folder:str):
    
    execute_notebook_component = components.load_component_from_file(COMPONENT_YAML_PATH)
    execute_notebook_op = execute_notebook_component(
        project_id=project_id,
        execution_id=execution_id,
        input_notebook_file=input_notebook,
        output_notebook_folder=output_notebook_folder,
        block_pipeline=BLOCK_PIPELINE,
        fail_pipeline=FAIL_PIPELINE,
        parameters=f'PROJECT_ID={project_id},EXECUTION_ID={execution_id}'
    )

For additional details about:
- Google Cloud Notebook Executor template, see the [ExecutionTemplate API documentation](https://cloud.google.com/notebooks/docs/reference/rest/v1/ExecutionTemplate).
- Pipeline types, see [kfp.dsl.types](https://kubeflow-pipelines.readthedocs.io/en/latest/_modules/kfp/dsl/types.html).
- Notebook parameters, see [Papermill parameters](https://papermill.readthedocs.io/en/latest/usage-parameterize.html)

## Run pipeline

In [16]:
NOW = datetime.now().strftime("%Y%m%d_%H%M%S")
EXECUTION_ID=f'naas_{NOW}'

compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path=PACKAGE_PATH)

api_client = AIPlatformClient(
    project_id=PROJECT_ID, 
    region=REGION)

response = api_client.create_run_from_job_spec(
    PACKAGE_PATH,
    pipeline_root=PIPELINE_ROOT_PATH,
    parameter_values={
        'project_id': PROJECT_ID,
        'execution_id': EXECUTION_ID,
        'input_notebook': INPUT_NOTEBOOK_FILE,
        'output_notebook_folder': OUTPUT_NOTEBOOK_FOLDER
    })

/Users/matthieum/development/miniconda/lib/python3.8/site-packages/kfp/v2/google/client/client.py:171: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  warnings.warn(
